In [87]:
import pandas as pd
import numpy as np
import datetime
import math

pd.set_option('display.max_rows', None)

In [2]:
full_df = pd.read_csv('./dump.csv')

In [4]:
full_df['old_ts'] = full_df['ts']
full_df['ts'] = pd.to_datetime(full_df['old_ts'])
full_df_sorted = full_df.sort_values('ts')
full_df_sorted = full_df_sorted.reset_index(drop=True)

In [10]:
full_df_sorted[full_df_sorted.ts > datetime.datetime(2020, 12, 2)].head()

,ts,symbol,type,price,remaining,old_ts
3531184,2020-12-02 00:00:00.081,BTC,t,18801.94,0.194117,2020-12-02 00:00:00.081
3531185,2020-12-02 00:00:00.081,BTC,t,18804.48,0.100000,2020-12-02 00:00:00.081
3531186,2020-12-02 00:00:00.084,BTC,t,18801.93,0.040816,2020-12-02 00:00:00.084
3531187,2020-12-02 00:00:00.085,BTC,f,18778.86,0.376700,2020-12-02 00:00:00.085
3531188,2020-12-02 00:00:00.090,BTC,t,18801.33,0.574000,2020-12-02 00:00:00.09


In [8]:
full_df_sorted[full_df_sorted.ts < datetime.datetime(2021, 1, 1)].tail()

,ts,symbol,type,price,remaining,old_ts
12161722,2020-12-18 04:51:53.623,BTC,t,22952.63,2.000000,2020-12-18 04:51:53.623
12161723,2020-12-18 04:51:53.623,BTC,t,22952.63,1.837246,2020-12-18 04:51:53.623
12161724,2020-12-18 04:51:54.119,BTC,t,22953.31,0.122725,2020-12-18 04:51:54.119
12161725,2020-12-18 04:51:54.602,BTC,t,22953.31,0.122724,2020-12-18 04:51:54.602
12161726,2020-12-18 04:51:55.918,BTC,t,22953.30,0.011155,2020-12-18 04:51:55.918


In [14]:
flat_df = full_df_sorted.iloc[3531184:12161726]
flat_df = flat_df.reset_index(drop=True)

In [15]:
flat_df.head()

,ts,symbol,type,price,remaining,old_ts
0,2020-12-02 00:00:00.081,BTC,t,18801.94,0.194117,2020-12-02 00:00:00.081
1,2020-12-02 00:00:00.081,BTC,t,18804.48,0.100000,2020-12-02 00:00:00.081
2,2020-12-02 00:00:00.084,BTC,t,18801.93,0.040816,2020-12-02 00:00:00.084
3,2020-12-02 00:00:00.085,BTC,f,18778.86,0.376700,2020-12-02 00:00:00.085
4,2020-12-02 00:00:00.090,BTC,t,18801.33,0.574000,2020-12-02 00:00:00.09


In [5]:
orig_df = pd.read_csv('./dump_10.csv')

In [6]:
orig_df.head()

,ts,symbol,type,price,remaining
0,2020-11-25 17:47:53.866,BTC,t,19006.86,0.010322
1,2020-11-25 17:47:58.066,BTC,t,19006.73,2.000000
2,2020-11-25 17:47:58.101,BTC,t,19005.91,0.589000
3,2020-11-25 17:47:58.115,BTC,t,19005.80,2.000000
4,2020-11-25 17:47:58.116,BTC,f,19003.64,0.078986


In [7]:
s_df = orig_df.iloc[:1000000]
s_df.to_csv('dump_10.csv', index=False)

In [8]:
orig_df['old_ts'] = orig_df['ts']

In [25]:
orig_df['ts'] = pd.to_datetime(orig_df['old_ts'])
orig_df_sorted = orig_df.sort_values('ts')
orig_df_sorted = orig_df_sorted.reset_index(drop=True)

In [26]:
orig_df_sorted = orig_df_sorted.drop(np.arange(96))
# orig_df_sorted = orig_df_sorted.reset_index()
# (orig_df_sorted['old_ts'] == '1970-01-01 00:00:00')

In [28]:
orig_df_sorted = orig_df_sorted.reset_index(drop=True)

In [22]:
orig_df_sorted.to_csv('sorted_10.csv', index=False)

In [23]:
orig_df_sorted = pd.read_csv('sorted_10.csv')

In [29]:
orig_df_sorted.head()

,ts,symbol,type,price,remaining,old_ts
0,2020-11-25 17:47:13.024,BTC,t,18994.03,0.180000,2020-11-25 17:47:13.024
1,2020-11-25 17:47:13.025,BTC,f,18992.33,0.067192,2020-11-25 17:47:13.025
2,2020-11-25 17:47:13.052,BTC,f,18992.33,0.073345,2020-11-25 17:47:13.052
3,2020-11-25 17:47:14.769,BTC,f,18992.34,0.100000,2020-11-25 17:47:14.769
4,2020-11-25 17:47:14.770,BTC,f,18992.35,0.064448,2020-11-25 17:47:14.77


In [30]:
orig_df_sorted.iloc[1].ts - orig_df_sorted.iloc[0].ts

Timedelta('0 days 00:00:00.001000')

In [15]:
orig_df_sorted.shape

(999904, 6)

In [31]:
def max_gap(c):
        
    if c.name % 1000 == 0:
        print(c.name)
    
    if c.name == 0:
        return pd.Timedelta(0)
#     return c.ts - small_df_sorted.loc[c.name - 1].ts
    return c.ts - orig_df_sorted.loc[c.name - 1].ts

In [32]:
# small_df_sorted['interval'] = small_df_sorted.apply(max_gap, axis=1)
orig_df_sorted['interval'] = orig_df_sorted.apply(max_gap, axis=1)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [33]:
# small_df_sorted['interval'].max()
orig_df_sorted['interval'].max()

Timedelta('0 days 00:00:23.980000')

In [34]:
orig_df_sorted.to_csv('with_interval_10.csv')

In [16]:
glob_df = flat_df

In [88]:
def round_decimals_down(number:float, decimals:int=2):
    """
    Returns a value rounded down to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.floor(number)

    factor = 10 ** decimals
    return math.floor(number * factor) / factor

In [19]:
segment_len = pd.Timedelta(10, 'seconds').total_seconds()
total_segments = (glob_df.iloc[-1].ts - glob_df.iloc[0].ts).total_seconds() / segment_len

In [20]:
def find_current_price(cur_time):
    index = (glob_df.ts - cur_time).abs().idxmin()
    if glob_df.iloc[index].ts > cur_time:
        return glob_df.iloc[index-1].price
    else:
        return glob_df.iloc[index].price

In [22]:
def find_current_bid(cur_time):
    try:
        index = (glob_df[(glob_df.type == 'f') & (glob_df.ts < cur_time)].ts - cur_time).abs().idxmin()
        return glob_df.loc[index].price
    except ValueError:
        return None

In [21]:
def find_current_ask(cur_time):
    try:
        index = (glob_df[(glob_df.type == 't') & (glob_df.ts < cur_time)].ts - cur_time).abs().idxmin()
        return glob_df.loc[index].price
    except ValueError:
        return None

In [23]:
def min_slot(wallet, cur_price):
    if wallet.price.isna().sum() > 0:
        return (wallet[wallet.price.isna()].iloc[0].name), True
    else:
        try:
            return (wallet[wallet.avail > 0].price - cur_price).astype(np.float128).idxmin(), False
        except ValueError:
            # all slots full
            return -1, False

In [63]:
# def max_slot(wallet, cur_price):
#     if wallet.price.isna().sum() > 0:
#         return None
#     else:
#         try:
#             return (wallet[wallet.btc > 0].price - cur_price).astype(np.float128).idxmax()
#         except:
#             # all slots empty
#             return -1
def max_slot(wallet, cur_price):
    try:
#         return (wallet[wallet.btc > 0].price - cur_price).astype(np.float128).idxmax()
        return (cur_price - wallet[wallet.btc > 0].price).astype(np.float128).idxmax()
    except:
        #all slots empty
        return -1

In [79]:
def buy(wallet, slot, cur_ask):
    if wallet.loc[slot].avail > 0:
#         btc = (wallet.loc[slot, 'avail'] / cur_price) * 0.9965
        # 0.001 gemini maker fee
        # 0.0003 ask overcut
        btc = round(wallet.loc[slot, 'avail'] / cur_ask, 8) * 0.999 * 0.9997
        wallet.loc[slot, 'avail'] -= btc * cur_ask
        wallet.loc[slot, 'price'] = cur_ask
        wallet.loc[slot, 'btc'] = btc

In [84]:
def sell(wallet, slot, cur_bid):
    if wallet.loc[slot, 'btc'] > 0:
#         avail = wallet.loc[slot, 'btc'] * cur_price * 0.9965
        # 0.001 gemini maker fee
        # 0.0003 bid undercut
        s_usd = round_decimals_down(wallet.loc[slot, 'btc'] * cur_bid * 0.999 * 0.9997, 2)
        if s_usd >= max_avail:
            btc_for_max_avail = round_decimals_down(max_avail / (cur_bid * 0.999 * 0.9997), 8)
            wallet.loc[slot, 'excess'] += wallet.loc[slot, 'btc'] - btc_for_max_avail
            wallet.loc[slot, 'avail'] = max_avail
            wallet.loc[slot, 'price'] = cur_bid
            wallet.loc[slot, 'btc'] = 0.0

In [96]:
def should_buy(wallet, min_s, is_start, cur_ask):
    if cur_ask is None:
        return False
    
    if is_start:
        return True

#     return min_s > 0 and cur_price * 1.0035 < wallet.loc[min_s, 'price'] * 0.9965
    return min_s >= 0 and cur_ask * 1.0015 < wallet.loc[min_s, 'price'] * 0.9985

In [97]:
def should_sell(wallet, max_s, cur_bid):
#     print(max_s, cur_bid)
    
    if max_s is None or cur_bid is None:
        return False
    
#     print(max_s)
#     print('{:0.2f}\t{:0.2f}'.format(cur_bid * 0.9985, wallet.loc[max_s, 'price'] * 1.0015))
#     return max_s > 0 and cur_price * 0.9965 > wallet.loc[max_s, 'price'] * 1.0035
    return max_s >= 0 and cur_bid * 0.9985 > wallet.loc[max_s, 'price'] * 1.0015

In [85]:
def liquidate(wallet, cur_ask):
    return wallet.avail.sum() + wallet.btc.sum() * cur_ask + wallet.excess.sum() * cur_ask

In [55]:
start_balance = 1000.0
num_slots = 2000
max_avail = start_balance / num_slots

def init_wallet():
    wallet = pd.DataFrame(columns=['price', 'avail', 'btc', 'excess'])
    
    for i in range(num_slots):
        wallet.loc[i] = {'price': None, 'avail': start_balance / num_slots, 'btc': 0.0, 'excess': 0.0}
        
    return wallet

In [89]:
log = []
wallet = init_wallet()

start_seg = 0

for seg_i in range(int(total_segments)):
    cur_time = glob_df.iloc[start_seg].ts + pd.Timedelta(seg_i * segment_len, 'seconds')
#     cur_price = find_current_price(cur_time)
    cur_bid = find_current_bid(cur_time)
    cur_ask = find_current_ask(cur_time)
    
    min_s, is_start = min_slot(wallet, cur_ask)
    if should_buy(wallet, min_s, is_start, cur_ask):
        log.append('buy: {}\t{}\t{}'.format(min_s, wallet.loc[min_s].price, cur_ask))
        buy(wallet, min_s, cur_ask)
    
    max_s = max_slot(wallet, cur_bid)
    if should_sell(wallet, max_s, cur_bid):
        log.append('sell: {}\t{}\t{}'.format(max_s, wallet.loc[max_s].price, cur_bid))
        sell(wallet, max_s, cur_bid)
    
    if seg_i > start_seg + 100000:
        break
        
    if seg_i % 100 == 0 and seg_i > 0:
        print(seg_i)
        print(cur_ask)
        print(wallet.excess.sum())
        print(liquidate(wallet, cur_ask))

100
18596.51
8.894598580000182e-07
999.8888937733528
200
18851.75
7.848579926000095e-06
1000.330259287897
300
18892.42
1.0939512979000103e-05
1000.3559837460083
400
18759.57
1.0939512979000103e-05
999.7138584933717
500
18788.33
1.1631382048000113e-05
999.9033632767664
600
18905.22
1.4710512859000073e-05
1000.978932983416
700
18879.58
2.016322531600014e-05
1000.621940157896
800
18825.14
2.016322531600014e-05
999.9727932639877
900
18788.16
2.018864029900014e-05
999.3812675672989
1000
18687.03
2.018864029900014e-05
997.5800445794234
1100
18738.62
2.0505943634000164e-05
998.6187269150063
1200
18607.48
2.321706569100024e-05
995.669690713368
1300
18664.48
2.409721254800029e-05
997.0627710146846
1400
18593.22
2.5870692369000313e-05
995.1565183598276
1500
18540.33
2.852594975200036e-05
993.644695307926
1600
18370.9
2.8590544082000362e-05
988.5588295504704
1700
18496.76
3.081057276800037e-05
992.5483245140471
1800
18571.17
3.4052139853000456e-05
994.9051722476071
1900
18629.48
3.952351025700050

KeyboardInterrupt: 

In [93]:
for e in log:
    print(e)

buy: 0	None	18792.87
buy: 1	None	18787.01
buy: 2	None	18786.9
buy: 3	None	18779.26
buy: 4	None	18779.26
buy: 5	None	18779.26
buy: 6	None	18778.13
buy: 7	None	18773.81
buy: 8	None	18765.1
buy: 9	None	18755.81
buy: 10	None	18737.01
buy: 11	None	18732.83
buy: 12	None	18725.37
buy: 13	None	18723.99
buy: 14	None	18736.14
buy: 15	None	18725.18
buy: 16	None	18691.44
buy: 17	None	18672.68
buy: 18	None	18657.35
buy: 19	None	18631.59
buy: 20	None	18647.31
buy: 21	None	18684.88
buy: 22	None	18660.44
buy: 23	None	18625.41
buy: 24	None	18635.65
buy: 25	None	18647.19
buy: 26	None	18684.01
buy: 27	None	18676.53
buy: 28	None	18676.89
buy: 29	None	18668.13
buy: 30	None	18628.02
buy: 31	None	18606.73
buy: 32	None	18607.69
buy: 33	None	18614.85
buy: 34	None	18576.41
buy: 35	None	18565.06
buy: 36	None	18561.71
buy: 37	None	18576.24
buy: 38	None	18527.72
buy: 39	None	18509.76
buy: 40	None	18520.9
buy: 41	None	18542.78
buy: 42	None	18543.72
buy: 43	None	18562.48
buy: 44	None	18574.38
buy: 45	None	18588.3
bu

buy: 1866	None	18621.96
sell: 1722	18482.42	18613.5
buy: 1867	None	18624.65
sell: 1681	18482.95	18616.4
buy: 1868	None	18615.35
sell: 1554	18483.23	18615.34
buy: 1869	None	18618.81
sell: 1555	18483.31	18611.71
buy: 1870	None	18618.45
sell: 1723	18485.21	18607.23
buy: 1871	None	18617.29
sell: 1679	18487.03	18608.26
buy: 1872	None	18611.19
sell: 1698	18488.05	18607.22
buy: 1873	None	18598.83
sell: 1677	18488.59	18586.19
buy: 1874	None	18615.32
sell: 1553	18488.74	18605.92
buy: 1875	None	18625.65
sell: 1709	18488.99	18610.35
buy: 1876	None	18626.58
sell: 1559	18489.0	18615.08
buy: 1877	None	18623.48
sell: 1684	18489.18	18619.28
buy: 1878	None	18626.58
sell: 1853	18490.22	18624.49
buy: 1879	None	18638.13
sell: 1552	18490.68	18627.28
buy: 1880	None	18644.59
sell: 1700	18491.51	18640.48
buy: 1881	None	18642.58
sell: 1719	18492.34	18641.42
buy: 1882	None	18644.02
sell: 1852	18493.35	18632.37
buy: 1883	None	18640.99
sell: 1850	18493.55	18632.47
buy: 1884	None	18627.63
sell: 1724	18494.0	18621.

In [90]:
wallet.iloc[:100]

,price,avail,btc,excess
0,18792.87,0.000572,0.000027,0.000000e+00
1,18866.81,0.500000,0.000000,4.541498e-08
2,18858.47,0.500000,0.000000,3.541498e-08
3,18836.94,0.500000,0.000000,2.538899e-08
4,18836.99,0.500000,0.000000,2.538899e-08
5,18836.99,0.500000,0.000000,2.538899e-08
6,18846.82,0.500000,0.000000,3.538899e-08
7,18845.29,0.500000,0.000000,3.538899e-08
8,18832.99,0.500000,0.000000,3.536300e-08
9,18839.50,0.500000,0.000000,5.535000e-08


In [95]:
max_slot(wallet, 19248)

0

In [94]:
cur_bid

19248.2